In [ ]:
#!pip install pyserini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.7 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.7 MB/s eta 0:00:00
  Created w

In [130]:
# 打开文件并读取内容
with open('/content/IR/Pubmed 220 Train 2022-01-21.txt', 'r') as f:
    content = f.read()

# 根据 '###' 分割文本内容
data_text = content.split('###')

# 初始化一个空列表用于存储结果
dataset = []
data_content = {}

# 遍历分割后的文本内容
for i in data_text[1:]:

    # 获取 id
    id = i[:i.find('\n')]
    # 获取 content，去除制表符和空行并拼接为一个字符串
    string = i[i.find('\n'):].split('\n')
    content = "".join([s[s.find('\t')+1:].split('\t')[0] for s in string if s.strip()])
    # 将 id 和 content 放入字典中，并添加到 dataset 列表中
    dataset.append({"id": id, "contents": content})

    data_content[id] = content

'''
for data in dataset:
    print(data)
'''

'\nfor data in dataset:\n    print(data)\n'

In [101]:
import json
with open('/content/IR/corups/retriver.json', 'w') as json_file:
    json.dump(dataset, json_file)

In [102]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/IR/corups \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2024-03-28 14:44:40,490 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:204) - Setting log level to INFO
2024-03-28 14:44:40,493 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:207) - ============ Loading Index Configuration ============
2024-03-28 14:44:40,493 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - AbstractIndexer settings:
2024-03-28 14:44:40,494 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) -  + DocumentCollection path: /content/IR/corups
2024-03-28 14:44:40,495 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + CollectionClass: JsonCollection
2024-03-28 14:44:40,498 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + Index path: indexes/sample_collection_jsonl
2024-03-28 14:44:40,498 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Threads: 1
2024-03-28 14:44:40,499 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Optimize (merge segments)? false
2024-03-28 

In [137]:
def Retreiver(keyword):
  from pyserini.search.lucene import LuceneSearcher
  import time

  start = time.time()
  searcher = LuceneSearcher('indexes/sample_collection_jsonl')
  hits = searcher.search(keyword, k = 3)
  end = time.time()

  print("執行時間 ：",(end-start),"秒")
  for i in range(len(hits)):
    tmp = []
    tmp.append( {"id":hits[i].docid,"content":data_content[hits[i].docid]} )
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')
    print(tmp[0])


result = Retreiver("cancer")

執行時間 ： 0.010490179061889648 秒
1 24747090 2.50920
{'id': '24747090', 'content': 'We examine the role of body mass index in the assessment of prostate cancer risk .A total of 3,258 participants who underwent biopsy ( including 1,902 men with a diagnosis of prostate cancer ) were identified from the Selenium and Vitamin E Cancer Prevention Trial .The associations of body mass index with prostate cancer and high grade prostate cancer were examined using logistic regression , adjusting for age , race , body mass index adjusted prostate specific antigen , digital rectal examination , family history of prostate cancer , biopsy history , prostate specific antigen velocity , and time between study entry and the last biopsy .The prediction models were compared with our previously developed body mass index adjusted Prostate Cancer Prevention Trial prostate cancer risk calculator .Of the study subjects 49.1 % were overweight and 29.3 % were obese .After adjustment , among men without a known famil

In [99]:
start = time.time()
keyword = "cancer"
result_list = []
for i in dataset:
  if keyword in i["contents"]:
    result_list.append(i)
end = time.time()
print("執行時間 : %f 秒"%(end-start))
for i in result_list[:10]:
  print(i)

執行時間 : 0.569539 秒
{'id': '16308853', 'content': 'Use of blue dye alone as a marker for sentinel lymph node ( SLN ) biopsy is effective , but combining it with isotope marking can improve the success rate .Use of the isotope adds extra cost and there are potential radiation hazards .The two techniques were compared in a randomized trial .Women with early breast cancer ( less than 3 cm ) and no palpable axillary nodes were recruited .Women older than 70 years with multicentric cancers or previous surgery to the breast or axilla were excluded .Patients were randomized to either blue dye alone or combined mapping for SLN biopsy .All women had a level I and II axillary dissection after the SLN biopsy .A total of 123 patients were recruited , of whom five were excluded from analysis .Blue dye alone was used in 57 women and 61 had combined mapping .Baseline demographic data were similar in the two cohorts .The success rate of SLN biopsy was higher with combined mapping than with blue dye alon